# Raw SQL x ORM Benchmarks
*Vojtěch Lengál*

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Intro
In this experiment, we're going to evaluate performance overhead of using [EF Core](https://learn.microsoft.com/en-us/ef/) (ORM framework for .NET) compared to raw SQL queries.


- SQLite (embedded DB)
- SQL Server

### Expected results:

## Testing data

Our data model represents a simple e-shop with the following entities:
- Category
- Product
- Order
- Customer

Here's a UML model of our data:

![](./diagrams/data_model.png)

I generated the test data, i.e.
- 1,000 Categories
- 1,000,000 Products
- 100,000 Orders (each order has <= 10 products)
- 1,000 Customers

using [Bogus](https://github.com/bchavez/Bogus) library.

So, all of the following benchmarks are ran on these data.

## Benchmarks

## Build & run

## Conclusions